In [1]:
from dotenv import load_dotenv
import os
from pathlib import Path

CURRENT_DIRECTORY_NOTEBOOK = None


def intitate_notebook():
    load_dotenv()
    global CURRENT_DIRECTORY_NOTEBOOK
    if CURRENT_DIRECTORY_NOTEBOOK is None:
        os.chdir(os.getenv("PROJECT_BASE_PATH"))
        CURRENT_DIRECTORY_NOTEBOOK = Path(os.getcwd())
        print("Current directory for notebook: ", CURRENT_DIRECTORY_NOTEBOOK)
    else:
        print(
            "Current directory for notebook is already set: ",
            CURRENT_DIRECTORY_NOTEBOOK,
        )


intitate_notebook()

Current directory for notebook:  /Users/shirshmall/Personal_Drive/Credit_Risk_MLOps_Project


In [2]:
import pandas as pd
import xgboost as xgb
from sklearn.metrics import (
    accuracy_score,
    precision_score,
    recall_score,
    f1_score,
    roc_auc_score,
    confusion_matrix,
    classification_report,
)

In [3]:
df = pd.read_parquet(
    "notebooks/training_initial_model/initial_processed_data/application_train.parquet"
)
df

,main_split_id,sk_id_curr,target,cnt_children,amt_income_total,amt_credit,amt_annuity,amt_goods_price,region_population_relative,days_birth,...,flag_own_car_string_indexed,flag_own_realty_string_indexed,name_type_suite_index,name_income_type_grouped_index,name_education_type_string_indexed,name_family_status_string_indexed,name_housing_type_string_indexed,occupation_type_index,weekday_appr_process_start_index,organization_type_index
0,12,456078,0,0,112500.0,390960.0,21339.0,337500.0,0.035792,-8573,...,0.0,1.0,0,0,4.0,3.0,1.0,-1,6,2
1,12,456079,0,0,148500.0,157050.0,6781.5,112500.0,0.030755,-21569,...,0.0,0.0,6,0,1.0,1.0,1.0,0,2,7
2,12,456080,0,0,121500.0,448056.0,17019.0,315000.0,0.019689,-18248,...,0.0,1.0,0,0,4.0,1.0,1.0,11,2,2
3,12,456081,0,0,72000.0,547344.0,19791.0,472500.0,0.022800,-15265,...,1.0,1.0,0,0,4.0,1.0,1.0,10,4,2
4,12,456082,0,0,157500.0,1078200.0,28570.5,900000.0,0.007020,-11180,...,1.0,0.0,0,3,1.0,3.0,1.0,0,3,17
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
307506,12,456072,0,1,135000.0,269550.0,21739.5,225000.0,0.006305,-8205,...,0.0,0.0,0,0,2.0,3.0,1.0,7,2,3
307507,12,456073,0,0,135000.0,755190.0,52690.5,675000.0,0.007330,-22525,...,0.0,1.0,6,2,4.0,0.0,1.0,-1,3,1
307508,12,456074,0,1,315000.0,730017.0,30928.5,652500.0,0.006629,-20243,...,1.0,0.0,0,0,4.0,5.0,2.0,16,5,0
307509,12,456075,0,0,112500.0,717003.0,21091.5,598500.0,0.032561,-22732,...,1.0,1.0,0,2,4.0,1.0,1.0,-1,5,1


In [4]:
df["main_split_id"].unique()

array([12,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11])

In [5]:
columns_to_drop = [
    "main_split_id",
    "sk_id_curr",
    "fondkapremont_mode",
    "housetype_mode",
    "emergencystate_mode",
    "wallsmaterial_mode",
    "target",
]


X_train = df[df["main_split_id"] <= 10]
y_train = X_train["target"]
X_train.drop(
    columns=columns_to_drop,
    inplace=True,
)
X_test = df[df["main_split_id"] > 10]
y_test = X_test["target"]
X_test.drop(
    columns=columns_to_drop,
    inplace=True,
)

/var/folders/x8/4hcwkc1s5nq77x8vmnjdj8900000gn/T/ipykernel_77401/3347119042.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_train.drop(
/var/folders/x8/4hcwkc1s5nq77x8vmnjdj8900000gn/T/ipykernel_77401/3347119042.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_test.drop(


In [6]:
# Initialize and train the XGBoost model
model = xgb.XGBClassifier()

model.fit(X_train, y_train)

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              feature_weights=None, gamma=None, grow_policy=None,
              importance_type=None, interaction_constraints=None,
              learning_rate=None, max_bin=None, max_cat_threshold=None,
              max_cat_to_onehot=None, max_delta_step=None, max_depth=None,
              max_leaves=None, min_child_weight=None, missing=nan,
              monotone_constraints=None, multi_strategy=None, n_estimators=None,
              n_jobs=None, num_parallel_tree=None, ...)

In [7]:
# Make predictions for both train and test
y_train_pred = model.predict(X_train)
y_test_pred = model.predict(X_test)
y_train_proba = model.predict_proba(X_train)[:, 1]
y_test_proba = model.predict_proba(X_test)[:, 1]


# Calculate metrics for both sets
def calculate_metrics(y_true, y_pred, y_proba, set_name):
    metrics = {
        f"Accuracy ({set_name})": accuracy_score(y_true, y_pred),
        f"Precision ({set_name})": precision_score(y_true, y_pred),
        f"Recall ({set_name})": recall_score(y_true, y_pred),
        f"F1 Score ({set_name})": f1_score(y_true, y_pred),
        f"ROC AUC ({set_name})": roc_auc_score(y_true, y_proba),
    }
    return metrics


train_metrics = calculate_metrics(y_train, y_train_pred, y_train_proba, "Train")
test_metrics = calculate_metrics(y_test, y_test_pred, y_test_proba, "Test")

# Combine metrics into a single DataFrame
metrics_df = pd.DataFrame([train_metrics, test_metrics]).T
metrics_df.columns = ["Train Set", "Test Set"]
metrics_df = metrics_df.applymap(lambda x: f"{x:.4f}")

# Display metrics
print("=" * 60)
print("XGBoost Classification Metrics - Train vs Test Comparison")
print("=" * 60)
print(metrics_df)
print("\n")

# Confusion matrices
print("Confusion Matrices:")
train_conf_matrix = confusion_matrix(y_train, y_train_pred)
test_conf_matrix = confusion_matrix(y_test, y_test_pred)

print("\nTraining Set Confusion Matrix:")
print(
    pd.DataFrame(
        train_conf_matrix,
        index=["Actual Negative", "Actual Positive"],
        columns=["Predicted Negative", "Predicted Positive"],
    )
)

print("\nTest Set Confusion Matrix:")
print(
    pd.DataFrame(
        test_conf_matrix,
        index=["Actual Negative", "Actual Positive"],
        columns=["Predicted Negative", "Predicted Positive"],
    )
)
print("\n")

# Classification reports
print("Classification Reports:")
print("\nTraining Set:")
print(classification_report(y_train, y_train_pred, target_names=["Class 0", "Class 1"]))

print("\nTest Set:")
print(classification_report(y_test, y_test_pred, target_names=["Class 0", "Class 1"]))

XGBoost Classification Metrics - Train vs Test Comparison
                  Train Set Test Set
Accuracy (Train)     0.9232      nan
Precision (Train)    0.8019      nan
Recall (Train)       0.0700      nan
F1 Score (Train)     0.1287      nan
ROC AUC (Train)      0.8583      nan
Accuracy (Test)         nan   0.9203
Precision (Test)        nan   0.4684
Recall (Test)           nan   0.0338
F1 Score (Test)         nan   0.0630
ROC AUC (Test)          nan   0.7543


Confusion Matrices:

Training Set Confusion Matrix:
                 Predicted Negative  Predicted Positive
Actual Negative              233853                 357
Actual Positive               19204                1445

Test Set Confusion Matrix:
                 Predicted Negative  Predicted Positive
Actual Negative               48316                 160
Actual Positive                4035                 141


Classification Reports:

Training Set:
              precision    recall  f1-score   support

     Class 0       0.

/var/folders/x8/4hcwkc1s5nq77x8vmnjdj8900000gn/T/ipykernel_77401/975630894.py:26: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  metrics_df = metrics_df.applymap(lambda x: f"{x:.4f}")
